In [57]:
# import required libraries
import numpy as np
import os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Dense, Dropout, Activation
import tensorflow as tf
tf.keras.utils.set_random_seed(1)

In [58]:
# loading files
from google.colab import files
uploaded = files.upload()

Saving BRCA.csv to BRCA (1).csv


In [59]:
# Reading csv into a dataframe and setting the index as the Patient ID
cancer_type_df = pd.read_csv('BRCA.csv').set_index('Patient_ID')
cancer_type_df.head()

,Age,Gender,Protein1,Protein2,Protein3,Protein4,Tumour_Stage,Histology,ER status,PR status,HER2 status,Surgery_type,Date_of_Surgery,Date_of_Last_Visit,Patient_Status
Patient_ID,,,,,,,,,,,,,,,
TCGA-D8-A1XD,36.0,FEMALE,0.080353,0.42638,0.54715,0.273680,III,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Modified Radical Mastectomy,15-Jan-17,19-Jun-17,Alive
TCGA-EW-A1OX,43.0,FEMALE,-0.420320,0.57807,0.61447,-0.031505,II,Mucinous Carcinoma,Positive,Positive,Negative,Lumpectomy,26-Apr-17,09-Nov-18,Dead
TCGA-A8-A079,69.0,FEMALE,0.213980,1.31140,-0.32747,-0.234260,III,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Other,08-Sep-17,09-Jun-18,Alive
TCGA-D8-A1XR,56.0,FEMALE,0.345090,-0.21147,-0.19304,0.124270,II,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Modified Radical Mastectomy,25-Jan-17,12-Jul-17,Alive
TCGA-BH-A0BF,56.0,FEMALE,0.221550,1.90680,0.52045,-0.311990,II,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Other,06-May-17,27-Jun-19,Dead


In [60]:
# Dropping rows that have empty cells
cancer_type_df.dropna(inplace=True)

In [61]:
# Checking amount of values in the Histology column
cancer_type_df['Histology'].value_counts()

Infiltrating Ductal Carcinoma     224
Infiltrating Lobular Carcinoma     81
Mucinous Carcinoma                 12
Name: Histology, dtype: int64

In [62]:
# Checking data types of the dataframe
cancer_type_df.dtypes

Age                   float64
Gender                 object
Protein1              float64
Protein2              float64
Protein3              float64
Protein4              float64
Tumour_Stage           object
Histology              object
ER status              object
PR status              object
HER2 status            object
Surgery_type           object
Date_of_Surgery        object
Date_of_Last_Visit     object
Patient_Status         object
dtype: object

In [63]:
# setting and instance of One Hot Encoder
enc = OneHotEncoder(sparse = False)

In [64]:
# Dropping un-wanted columns in the dataframe
cancer_type_df.drop(columns= ['Gender', 'Tumour_Stage', 'Surgery_type', "Date_of_Surgery", 'Date_of_Last_Visit', 'Patient_Status'], inplace=True)
cancer_type_df.head()

,Age,Protein1,Protein2,Protein3,Protein4,Histology,ER status,PR status,HER2 status
Patient_ID,,,,,,,,,
TCGA-D8-A1XD,36.0,0.080353,0.42638,0.54715,0.273680,Infiltrating Ductal Carcinoma,Positive,Positive,Negative
TCGA-EW-A1OX,43.0,-0.420320,0.57807,0.61447,-0.031505,Mucinous Carcinoma,Positive,Positive,Negative
TCGA-A8-A079,69.0,0.213980,1.31140,-0.32747,-0.234260,Infiltrating Ductal Carcinoma,Positive,Positive,Negative
TCGA-D8-A1XR,56.0,0.345090,-0.21147,-0.19304,0.124270,Infiltrating Ductal Carcinoma,Positive,Positive,Negative
TCGA-BH-A0BF,56.0,0.221550,1.90680,0.52045,-0.311990,Infiltrating Ductal Carcinoma,Positive,Positive,Negative


In [65]:
# creating a new dataframe for the features of the analysis
cancer_feat = cancer_type_df.copy().drop(columns = ['Histology'])
cancer_feat.head()

,Age,Protein1,Protein2,Protein3,Protein4,ER status,PR status,HER2 status
Patient_ID,,,,,,,,
TCGA-D8-A1XD,36.0,0.080353,0.42638,0.54715,0.273680,Positive,Positive,Negative
TCGA-EW-A1OX,43.0,-0.420320,0.57807,0.61447,-0.031505,Positive,Positive,Negative
TCGA-A8-A079,69.0,0.213980,1.31140,-0.32747,-0.234260,Positive,Positive,Negative
TCGA-D8-A1XR,56.0,0.345090,-0.21147,-0.19304,0.124270,Positive,Positive,Negative
TCGA-BH-A0BF,56.0,0.221550,1.90680,0.52045,-0.311990,Positive,Positive,Negative


In [66]:
# creating a list containing the column names of object data types
object_variables = list(cancer_feat.dtypes[cancer_feat.dtypes == "object"].index)
object_variables

['ER status', 'PR status', 'HER2 status']

In [67]:
# Encoding the columns of object types
encoded_data = enc.fit_transform(cancer_feat[object_variables])

In [68]:
# creating a dataframe of the encoded columns
encoded_df = pd.DataFrame(encoded_data,columns= enc.get_feature_names_out(object_variables))
encoded_df.head()

,ER status_Positive,PR status_Positive,HER2 status_Negative,HER2 status_Positive
0,1.0,1.0,1.0,0.0
1,1.0,1.0,1.0,0.0
2,1.0,1.0,1.0,0.0
3,1.0,1.0,1.0,0.0
4,1.0,1.0,1.0,0.0


In [69]:
# dropping columns in that were encoded
cancer_feat.drop(columns=object_variables, inplace=True)
cancer_feat.reset_index(inplace=True)
cancer_feat.head()

,Patient_ID,Age,Protein1,Protein2,Protein3,Protein4
0,TCGA-D8-A1XD,36.0,0.080353,0.42638,0.54715,0.273680
1,TCGA-EW-A1OX,43.0,-0.420320,0.57807,0.61447,-0.031505
2,TCGA-A8-A079,69.0,0.213980,1.31140,-0.32747,-0.234260
3,TCGA-D8-A1XR,56.0,0.345090,-0.21147,-0.19304,0.124270
4,TCGA-BH-A0BF,56.0,0.221550,1.90680,0.52045,-0.311990


In [70]:
# Creating a the features dataframe using the encoded dataframe and original feature dataframe
X = pd.concat([cancer_feat, encoded_df], axis=1).set_index('Patient_ID')
X.head()

,Age,Protein1,Protein2,Protein3,Protein4,ER status_Positive,PR status_Positive,HER2 status_Negative,HER2 status_Positive
Patient_ID,,,,,,,,,
TCGA-D8-A1XD,36.0,0.080353,0.42638,0.54715,0.273680,1.0,1.0,1.0,0.0
TCGA-EW-A1OX,43.0,-0.420320,0.57807,0.61447,-0.031505,1.0,1.0,1.0,0.0
TCGA-A8-A079,69.0,0.213980,1.31140,-0.32747,-0.234260,1.0,1.0,1.0,0.0
TCGA-D8-A1XR,56.0,0.345090,-0.21147,-0.19304,0.124270,1.0,1.0,1.0,0.0
TCGA-BH-A0BF,56.0,0.221550,1.90680,0.52045,-0.311990,1.0,1.0,1.0,0.0


In [71]:
# Setting the target variable
y = cancer_type_df['Histology']
y.value_counts()

Infiltrating Ductal Carcinoma     224
Infiltrating Lobular Carcinoma     81
Mucinous Carcinoma                 12
Name: Histology, dtype: int64

In [72]:
# Calculating the number of output and input nodes
number_of_classes = len(list(y.unique()))
number_of_predictors = len(X.columns)
print(number_of_classes)
print(number_of_predictors)

3
9


In [73]:
# Creating a Label Encoder instance and and transforming the target into an integer
encoder = LabelEncoder()
encoder.fit(y)
encoded_y = encoder.transform(y)
encoded_y

array([0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 2, 1, 2, 0, 0, 0, 0, 0, 0,
       0, 2, 2, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 2, 0, 0, 2, 0, 0,
       0, 1, 0, 0, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1,
       1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1,

In [74]:
# Transforming the encoded_y to binary
y_categorical = to_categorical(encoded_y, num_classes=number_of_classes)
y_categorical

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0

In [75]:
# Splitting the Features(X) and Target(y) into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, random_state = 1)

In [76]:
# Creating the Standard Scaler instance and scaling the Feature datasets
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [77]:
# Creating the Sequentail neural network instance
nn = Sequential()

In [78]:
# Creating the neural network layers
nn.add(Dense(25, input_dim = number_of_predictors, activation = 'relu'))
nn.add(Dense(25, activation = 'relu'))
nn.add(Dense(25, activation = 'tanh'))
nn.add(Dense(25, activation = 'tanh'))
nn.add(Dropout(.1))
nn.add(Dense(number_of_classes, activation='softmax'))

In [79]:
# Compiling the neural network using Categorical Crossentropy loss, the Adam optimizer, and Categorical Accuracy metric
nn.compile(loss="categorical_crossentropy",
              optimizer= "adam",
              metrics=['categorical_accuracy'])

In [80]:
# Displaying the makeup of the neural network
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 25)                250       
                                                                 
 dense_6 (Dense)             (None, 25)                650       
                                                                 
 dense_7 (Dense)             (None, 25)                650       
                                                                 
 dense_8 (Dense)             (None, 25)                650       
                                                                 
 dropout_1 (Dropout)         (None, 25)                0         
                                                                 
 dense_9 (Dense)             (None, 3)                 78        
                                                                 
Total params: 2278 (8.90 KB)
Trainable params: 2278 (8

In [81]:
# Training the neural network using 200 epochs
number_of_epochs = 200
nn.fit(X_train_scaled, y_train,
       epochs = number_of_epochs,
       shuffle = True)

Epoch 1/200
8/8 [==============================] - 1s 4ms/step - loss: 1.1789 - categorical_accuracy: 0.2025
Epoch 2/200
8/8 [==============================] - 0s 4ms/step - loss: 0.9574 - categorical_accuracy: 0.6118
Epoch 3/200
8/8 [==============================] - 0s 3ms/step - loss: 0.8351 - categorical_accuracy: 0.6962
Epoch 4/200
8/8 [==============================] - 0s 3ms/step - loss: 0.7960 - categorical_accuracy: 0.6835
Epoch 5/200
8/8 [==============================] - 0s 3ms/step - loss: 0.7707 - categorical_accuracy: 0.6751
Epoch 6/200
8/8 [==============================] - 0s 3ms/step - loss: 0.7710 - categorical_accuracy: 0.6793
Epoch 7/200
8/8 [==============================] - 0s 3ms/step - loss: 0.7664 - categorical_accuracy: 0.6793
Epoch 8/200
8/8 [==============================] - 0s 3ms/step - loss: 0.7472 - categorical_accuracy: 0.6751
Epoch 9/200
8/8 [==============================] - 0s 3ms/step - loss: 0.7505 - categorical_accuracy: 0.6751
Epoch 10/200
8/8 [=

In [82]:
# Evaluating the training
nn.evaluate(X_test_scaled,y_test, verbose=2)

3/3 - 0s - loss: 1.5331 - categorical_accuracy: 0.7125 - 189ms/epoch - 63ms/step


[1.5330744981765747, 0.7124999761581421]

In [83]:
# Using the testing data to predict on the model
predictions = nn.predict(X_test)
predictions

3/3 [==============================] - 0s 4ms/step


array([[9.99999583e-01, 1.06496948e-07, 2.01445687e-07],
       [9.99999583e-01, 9.89249074e-08, 2.23038739e-07],
       [9.99999702e-01, 1.19406138e-07, 1.65172096e-07],
       [9.99999583e-01, 9.94277016e-08, 2.26513066e-07],
       [9.99999583e-01, 9.91667193e-08, 2.26644232e-07],
       [9.99999583e-01, 8.54709228e-08, 2.84686053e-07],
       [9.99999583e-01, 1.04424167e-07, 2.10672411e-07],
       [9.99999464e-01, 8.10723932e-08, 3.07685156e-07],
       [9.99999702e-01, 1.19296416e-07, 1.72576350e-07],
       [9.99999583e-01, 8.59817249e-08, 2.80127040e-07],
       [9.99999464e-01, 7.79409817e-08, 3.29800628e-07],
       [9.99999464e-01, 7.98893538e-08, 3.16660532e-07],
       [9.99999464e-01, 7.78469413e-08, 3.30121281e-07],
       [9.99999583e-01, 9.87729436e-08, 2.28064195e-07],
       [9.99999464e-01, 8.15640675e-08, 3.02099920e-07],
       [9.99999583e-01, 1.11256298e-07, 1.91452429e-07],
       [9.99999583e-01, 9.21635035e-08, 2.51346961e-07],
       [9.99999583e-01, 8.75682

In [84]:
# Get the most likely prediction for each observation
most_likely = np.argmax(predictions, axis=1)
most_likely

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [85]:
# transforming the results back to their original values
most_likely = encoder.inverse_transform((most_likely))
most_likely

array(['Infiltrating Ductal Carcinoma', 'Infiltrating Ductal Carcinoma',
       'Infiltrating Ductal Carcinoma', 'Infiltrating Ductal Carcinoma',
       'Infiltrating Ductal Carcinoma', 'Infiltrating Ductal Carcinoma',
       'Infiltrating Ductal Carcinoma', 'Infiltrating Ductal Carcinoma',
       'Infiltrating Ductal Carcinoma', 'Infiltrating Ductal Carcinoma',
       'Infiltrating Ductal Carcinoma', 'Infiltrating Ductal Carcinoma',
       'Infiltrating Ductal Carcinoma', 'Infiltrating Ductal Carcinoma',
       'Infiltrating Ductal Carcinoma', 'Infiltrating Ductal Carcinoma',
       'Infiltrating Ductal Carcinoma', 'Infiltrating Ductal Carcinoma',
       'Infiltrating Ductal Carcinoma', 'Infiltrating Ductal Carcinoma',
       'Infiltrating Ductal Carcinoma', 'Infiltrating Ductal Carcinoma',
       'Infiltrating Ductal Carcinoma', 'Infiltrating Ductal Carcinoma',
       'Infiltrating Ductal Carcinoma', 'Infiltrating Ductal Carcinoma',
       'Infiltrating Ductal Carcinoma', 'Infiltrati

In [86]:
# Displaying count of predictions
pd.DataFrame(most_likely).value_counts()

Infiltrating Ductal Carcinoma    80
dtype: int64